In [1]:
import pandas as pd
import numpy as np

In [2]:
coupon_list_train = pd.read_csv('input/coupon_list_train.csv')
coupon_list_test = pd.read_csv('input/coupon_list_test.csv')
user_list = pd.read_csv('input/user_list.csv')
coupon_purchases_train = pd.read_csv("input/coupon_detail_train.csv")

coupon_area_train = pd.read_csv('input/coupon_area_train.csv')
coupon_area_test = pd.read_csv('input/coupon_area_test.csv')
coupon_visit_train = pd.read_csv('input/coupon_visit_train.csv')

In [3]:
coupon_list_train

,CAPSULE_TEXT,GENRE_NAME,PRICE_RATE,CATALOG_PRICE,DISCOUNT_PRICE,DISPFROM,DISPEND,DISPPERIOD,VALIDFROM,VALIDEND,...,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name,COUPON_ID_hash
0,グルメ,グルメ,50,3000,1500,2011-07-08 12:00:00,2011-07-09 12:00:00,1,2011-07-10,2011-12-08,...,1.0,0.0,0.0,1.0,1.0,0.0,関東,埼玉県,埼玉,6b263844241eea98c5a97f1335ea82af
1,グルメ,グルメ,51,2080,1000,2011-07-01 12:00:00,2011-07-02 12:00:00,1,2011-07-03,2011-12-04,...,1.0,1.0,1.0,1.0,1.0,1.0,関東,千葉県,千葉,cc031f250e8bad1e24060263b9fc0ddd
2,グルメ,グルメ,50,7000,3500,2011-07-12 12:00:00,2011-07-15 12:00:00,3,2011-07-16,2012-01-11,...,1.0,1.0,1.0,1.0,1.0,1.0,関東,千葉県,千葉,ba5e9b7453ca52ff711635a5d2e8102d
3,グルメ,グルメ,50,3000,1500,2011-07-09 12:00:00,2011-07-11 12:00:00,2,2011-07-12,2011-12-01,...,1.0,0.0,0.0,1.0,1.0,1.0,関東,千葉県,千葉,3e1ffbedca3569f9e8032d401e8cb4e6
4,グルメ,グルメ,50,2000,1000,2011-07-05 12:00:00,2011-07-06 12:00:00,1,2011-07-07,2011-12-30,...,1.0,0.0,0.0,1.0,1.0,0.0,関東,千葉県,千葉,782934b6c815b4030ea204eef7d4a734
5,グルメ,グルメ,50,4000,2000,2011-07-24 12:00:00,2011-07-26 12:00:00,2,2011-07-27,2011-10-30,...,1.0,1.0,1.0,1.0,1.0,1.0,関東,千葉県,千葉,439db4336f84d4e4a13290fc0d7e6c4e
6,グルメ,グルメ,51,3495,1700,2011-09-03 12:00:00,2011-09-05 12:00:00,2,2011-09-07,2012-02-28,...,1.0,1.0,1.0,1.0,1.0,1.0,関東,東京都,新宿・高田馬場・中野・吉祥寺,c9144a97ed61cb778414b758c704d828
7,グルメ,グルメ,51,3100,1500,2011-08-02 12:00:00,2011-08-04 12:00:00,2,2011-08-05,2012-01-31,...,1.0,1.0,1.0,0.0,1.0,1.0,関西,京都府,京都,2b00d8dfc1a9fcf58e5014cbfd4a1e48
8,グルメ,グルメ,50,3500,1750,2011-08-11 12:00:00,2011-08-13 12:00:00,2,2011-08-14,2011-11-14,...,1.0,1.0,1.0,1.0,1.0,1.0,関東,東京都,恵比寿・目黒・品川,74e3a12659d38ef20378f156cfeaf204
9,グルメ,グルメ,57,10500,4500,2011-09-21 12:00:00,2011-09-24 12:00:00,3,2011-09-26,2012-02-29,...,0.0,0.0,1.0,0.0,0.0,1.0,関東,東京都,銀座・新橋・東京・上野,8a0ab8c5dd5014df8b4860233238ed44


In [4]:
coupon_purchases_train

,ITEM_COUNT,I_DATE,SMALL_AREA_NAME,PURCHASEID_hash,USER_ID_hash,COUPON_ID_hash
0,1,2012-03-28 15:06:06,兵庫,c820a8882374a4e472f0984a8825893f,d9dca3cb44bab12ba313eaa681f663eb,34c48f84026e08355dc3bd19b427f09a
1,1,2011-07-04 23:52:54,銀座・新橋・東京・上野,1b4eb2435421ede98c8931c42e8220ec,560574a339f1b25e57b0221e486907ed,767673b7a777854a92b73b0934ddfae7
2,1,2011-07-16 00:52:49,恵比寿・目黒・品川,36b5f9ba46c44b65587d0b16f2e4c77f,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
3,1,2011-07-16 00:54:53,恵比寿・目黒・品川,2f30f46937cc9004774e576914b2aa1a,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
4,1,2011-07-16 00:55:52,恵比寿・目黒・品川,4d000c64a55ac573d0ae1a8f03677f50,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
5,1,2011-07-16 00:57:09,恵比寿・目黒・品川,d8b030c8a4a2c1051997092a2985b373,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
6,2,2011-07-16 00:58:29,恵比寿・目黒・品川,2c98138766edf5d5be97ed96aa188c7d,560574a339f1b25e57b0221e486907ed,4f3b5b91d9831192557c056022fdc1f2
7,2,2011-09-07 00:04:47,渋谷・青山・自由が丘,5a45d62c72a4cc4155786b03bec74c96,560574a339f1b25e57b0221e486907ed,259a22533cab381464c89f616a56d12d
8,1,2011-10-19 22:21:50,渋谷・青山・自由が丘,592bb67c0553648d6ebd99a4442606f8,560574a339f1b25e57b0221e486907ed,f4a6f861a266799b07cd99c26e80962b
9,1,2011-12-01 16:53:45,恵比寿・目黒・品川,509883c481716b1f3abeac8372b36a5d,560574a339f1b25e57b0221e486907ed,4257a8a169b40bbbcb002ec47ab3de13


In [5]:
purchased_coupons_train = coupon_purchases_train.merge(coupon_list_train, on='COUPON_ID_hash',
                                                 how='inner')

In [6]:
purchased_coupons_train

,ITEM_COUNT,I_DATE,SMALL_AREA_NAME,PURCHASEID_hash,USER_ID_hash,COUPON_ID_hash,CAPSULE_TEXT,GENRE_NAME,PRICE_RATE,CATALOG_PRICE,...,USABLE_DATE_WED,USABLE_DATE_THU,USABLE_DATE_FRI,USABLE_DATE_SAT,USABLE_DATE_SUN,USABLE_DATE_HOLIDAY,USABLE_DATE_BEFORE_HOLIDAY,large_area_name,ken_name,small_area_name
0,1,2012-03-28 15:06:06,兵庫,c820a8882374a4e472f0984a8825893f,d9dca3cb44bab12ba313eaa681f663eb,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
1,1,2012-03-26 23:26:24,千葉,028ac37f96a17e6be2db51ed0884ac5a,dd8a79645e8170db6a91bad1d89b4055,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
2,1,2012-03-26 13:50:20,新宿・高田馬場・中野・吉祥寺,eb7f867c9f1b14c1a2b1cc47004828ae,658249379aaaf2b2320061e5218c0c2c,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
3,1,2012-03-28 22:46:58,和歌山,fddda2bdaba87fc9b86d97e908152014,1c4aad19f033d97dc163b481116de322,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
4,1,2012-03-29 15:09:51,宮崎,6659c8fdb5f66b4013613ff770f91db7,e78ae0887c4a9dce91ff0d2a45a9f72c,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
5,1,2012-03-29 12:42:37,千葉,be5af03efe7e4bc2f1e7e8317531cd74,39cabe60f73ea8cad96d7f895ea944ee,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
6,1,2012-03-28 22:45:16,和歌山,aae8c29627af3489c530f141eb02a3a1,fca1e37548c9ca5415a9850a1b9b2c28,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
7,1,2012-03-29 18:10:28,栃木,ea4cf60407a03246be6c588c12542780,3511c63ae9ce45fc3b5cf5054956d5a4,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
8,1,2012-03-28 00:05:32,栃木,448cec2c62179725c9ac0d93006074d0,77b92939627690028e22834d7920bf4e,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺
9,2,2012-03-27 11:47:59,栃木,f8f426b0f2beb05978e856a727febac8,1ce49ceb6f55d3b3ed3180f811ec3ffb,34c48f84026e08355dc3bd19b427f09a,宅配,宅配,78,7200,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,関東,東京都,新宿・高田馬場・中野・吉祥寺


In [7]:
features = ['COUPON_ID_hash', 'USER_ID_hash',
            'GENRE_NAME', 'DISCOUNT_PRICE', 'PRICE_RATE',
            'USABLE_DATE_MON', 'USABLE_DATE_TUE', 'USABLE_DATE_WED', 'USABLE_DATE_THU',
            'USABLE_DATE_FRI', 'USABLE_DATE_SAT', 'USABLE_DATE_SUN', 'USABLE_DATE_HOLIDAY',
            'USABLE_DATE_BEFORE_HOLIDAY', 'large_area_name', 'ken_name', 'small_area_name']

In [8]:
purchased_coupons_train = purchased_coupons_train[features]